In [2]:
from pyecharts.charts import Radar
from pyecharts import options as opts
import pandas as pd
import numpy as np

# 1. 读取数据
df = pd.read_excel("grade.xlsx")

# 2. 构造五大能力维度
df['听力理解能力'] = df[['新闻报道', '日常对话', '日常短文']].mean(axis=1)
df['阅读理解能力'] = df[['阅读1', '阅读2', '阅读3', '阅读4']].mean(axis=1)
df['语言知识与应用'] = df[['词汇知识', '语法知识', '语言应用']].mean(axis=1)
df['口头表达能力'] = df[['语音语调准确度   &              话语连贯流利度',
                       '语言准确度  &          可理解度',
                       '话题内容广度    &          话题展开度 ',
                       '交际互动活跃度     &           语言适切度']].mean(axis=1)
df['书面表达能力'] = df[['内容', '结构', '语言', '交互']].mean(axis=1)

# 3. 聚合：每个学院五大能力平均分
ability_cols = ['听力理解能力', '阅读理解能力', '语言知识与应用', '口头表达能力', '书面表达能力']
college_abilities = df.groupby("学院")[ability_cols].mean()

# 4. 构造雷达图指标
schema = [
    {"name": "听力理解", "max": 10},
    {"name": "阅读理解", "max": 10},
    {"name": "语言知识", "max": 10},
    {"name": "口头表达", "max": 10},
    {"name": "书面表达", "max": 10},
]

colors = ["#5470C6", "#91CC75", "#EE6666", "#73C0DE", "#3BA272",
          "#FC8452", "#9A60B4", "#EA7CCC", "#FAC858", "#6B8E23",
          "#FF8C00", "#B22222", "#1E90FF"]

r = Radar(init_opts=opts.InitOpts(width="1000px", height="700px"))
r.add_schema(schema=schema)

for idx, (school, row) in enumerate(college_abilities.iterrows()):
    r.add(
        series_name=school,
        data=[row.tolist()],
        linestyle_opts=opts.LineStyleOpts(width=2, color=colors[idx % len(colors)]),
        label_opts=opts.LabelOpts(is_show=False)
    )

r.set_global_opts(
    title_opts=opts.TitleOpts(title="各学院能力雷达图", pos_top="2%"),
    legend_opts=opts.LegendOpts(type_="scroll", pos_top="10%", pos_left="center"),
)


# 9. 输出文件
r.render("各学院五项能力雷达图.html")
print(" 成功生成文件：各学院五项能力雷达图.html")

 成功生成文件：各学院五项能力雷达图.html


In [7]:
from pyecharts.charts import Bar, Line, Grid
from pyecharts import options as opts
import pandas as pd
import numpy as np

# 读取数据
df = pd.read_excel("grade.xlsx")
scores = df["原始总分"].dropna()

# 统计直方图分布
bins = np.histogram_bin_edges(scores, bins=15)
hist, edges = np.histogram(scores, bins=bins)
x = [f"{int(edges[i])}~{int(edges[i+1])}" for i in range(len(edges)-1)]

# 累计百分比（用于折线）
cum_percent = np.cumsum(hist) / np.sum(hist) * 100

# 创建柱状图
bar = (
    Bar()
    .add_xaxis(x)
    .add_yaxis("学生人数", hist.tolist(), yaxis_index=0, label_opts=opts.LabelOpts(is_show=False))
    .extend_axis(  # 添加第二个 Y 轴（右侧）
        yaxis=opts.AxisOpts(
            name="累计百分比",
            type_="value",
            min_=0,
            max_=100,
            axislabel_opts=opts.LabelOpts(formatter="{value}%")
        )
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="原始总分分布图（含累计曲线）"),
        xaxis_opts=opts.AxisOpts(name="原始总分", axislabel_opts={"rotate": 45}),
        yaxis_opts=opts.AxisOpts(name="学生人数"),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        legend_opts=opts.LegendOpts(pos_top="5%")
    )
)

# 创建折线图（累计百分比）
line = (
    Line()
    .add_xaxis(x)
    .add_yaxis("累计百分比", cum_percent.tolist(), yaxis_index=1, label_opts=opts.LabelOpts(is_show=False), z=5)

)

# 合并
bar.overlap(line)
bar.render("原始总分_改进版分布图.html")

'/Users/hemoqi/Documents/python_homework/数据可视化/pyecharts_ex/pyecharts_homework/原始总分_改进版分布图.html'

In [15]:
from pyecharts.charts import Scatter, Timeline, Line
from pyecharts import options as opts

timeline = Timeline(init_opts=opts.InitOpts(width="900px", height="600px"))

for dim in dimensions:
    x_data = df["原始总分"].tolist()
    y_data = df[dim].tolist()

    # 计算线性回归系数
    z = np.polyfit(x_data, y_data, 1)
    p = np.poly1d(z)
    reg_line = [[x, p(x)] for x in sorted(x_data)]

    # 计算相关系数
    r = np.corrcoef(x_data, y_data)[0, 1]
    subtitle = f"相关系数 r = {r:.2f}"

    scatter = (
        Scatter()
        .add_xaxis(x_data)
        .add_yaxis(dim, y_data, symbol_size=4, label_opts=opts.LabelOpts(is_show=False))
        .set_series_opts()
    )

    line = (
        Line()
        .add_xaxis([x for x, _ in reg_line])
        .add_yaxis("回归线", [y for _, y in reg_line], is_symbol_show=False,
                   linestyle_opts=opts.LineStyleOpts(type_="dashed", width=2))
    )

    chart = scatter.overlap(line).set_global_opts(
        title_opts=opts.TitleOpts(title=f"{dim} vs 原始总分", subtitle=subtitle),
        xaxis_opts=opts.AxisOpts(name="原始总分", min_=0, max_=120),
        yaxis_opts=opts.AxisOpts(name=dim, min_=0, max_=10),
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{b}: ({c})"),
    )

    timeline.add(chart, time_point=dim)

timeline.render("原始总分_五维度_动态散点图.html")
print(" 成功生成：原始总分_五维度_动态散点图.html")


 成功生成：原始总分_五维度_动态散点图.html


In [18]:
from pyecharts.charts import Bar
from pyecharts import options as opts

# 准备数据
ability_cols = ['听力理解能力', '阅读理解能力', '语言知识与应用', '口头表达能力', '书面表达能力']
college_abilities = df.groupby("学院")[ability_cols].mean()

# 横轴：学院列表
x = list(college_abilities.index)

# 每项能力作为一个系列（堆叠）
bar = Bar(init_opts=opts.InitOpts(width="1000px", height="600px"))
bar.add_xaxis(x)

for col in ability_cols:
    bar.add_yaxis(
        series_name=col,
        stack="总能力",  # stack 参数实现堆叠
        y_axis=college_abilities[col].round(2).tolist(),
        label_opts=opts.LabelOpts(is_show=False)
    )

# 图配置
bar.set_global_opts(
    title_opts=opts.TitleOpts(title="各学院五项能力构成对比（堆叠条形图）"),
    xaxis_opts=opts.AxisOpts(name="学院", axislabel_opts={"rotate": 30}),
    yaxis_opts=opts.AxisOpts(name="平均得分（堆叠）"),
    tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="shadow"),
    legend_opts=opts.LegendOpts(pos_top="5%")
)

bar.render("各学院能力堆叠条形图.html")
print("成功生成：各学院能力堆叠条形图.html")

成功生成：各学院能力堆叠条形图.html


In [19]:
from pyecharts.charts import Boxplot, Scatter, Grid
from pyecharts import options as opts

# 提取每个学院的口头表达能力得分
col_name = "口头表达能力"
grouped = df.groupby("学院")[col_name]
x = list(grouped.groups.keys())  # 学院列表
y_raw = [group.tolist() for _, group in grouped]

# 准备 boxplot 数据结构
boxplot = Boxplot()
y_box = boxplot.prepare_data(y_raw)

# 创建 boxplot
b = (
    Boxplot()
    .add_xaxis(x)
    .add_yaxis("箱线图", y_box)
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"各学院 {col_name} 分布（pyecharts改进版）"),
        xaxis_opts=opts.AxisOpts(name="学院", axislabel_opts={"rotate": 45}),
        yaxis_opts=opts.AxisOpts(name=col_name, min_=0, max_=10),
        tooltip_opts=opts.TooltipOpts(trigger="item", axis_pointer_type="shadow"),
        legend_opts=opts.LegendOpts(pos_top="5%"),
    )
)

# 创建分布点图（重现“分布轮廓”效果）
scatter = Scatter()
scatter.add_xaxis(x)
for i, group in enumerate(y_raw):
    scatter.add_yaxis("", group, xaxis_index=0, yaxis_index=0,
                      symbol_size=4, label_opts=opts.LabelOpts(is_show=False))

# 合并展示
b.overlap(scatter).render("各学院口头表达能力分布_改进版.html")
print("成功生成：各学院口头表达能力分布_改进版.html")

成功生成：各学院口头表达能力分布_改进版.html
